In [1]:
import pandas as pd

In [2]:
def format_res(df: pd.DataFrame, n_splits: int, cluster_outliers: str):

    precision_mean = df['precision'].mean()
    precision_se = df['precision'].sem()

    recall_mean = df['recall'].mean() 
    recall_se = df['recall'].sem()

    f1_mean = df['f1'].mean() 
    f1_se = df['f1'].sem()

    time_mean = df['time'].mean()
    time_se = df['time'].sem()

    output = (
        n_splits, 
        cluster_outliers,
        precision_mean,
        precision_se,
        recall_mean,
        recall_se,
        f1_mean,
        f1_se,
        time_mean,
        time_se
        )
    return output

In [3]:
def get_res(n_splits, cluster_outliers):
    df = pd.read_csv(f'res_1by1_{n_splits}_{cluster_outliers}.csv')
    return format_res(df, n_splits, cluster_outliers)

In [4]:
df_res = pd.DataFrame(columns=['n_splits', 'cluster_outliers', 'precision_mean', 'precision_se', 'recall_mean', 'recall_se', 'f1_mean', 'f1_se', 'time_mean', 'time_se'])

for n_splits in [50, 10, 100]:
    for cluster_outliers in ['all', 'new', 'skip']:
        df_res.loc[len(df_res)] = get_res(n_splits, cluster_outliers)

In [5]:
df_res.sort_values(by=['precision_mean', 'f1_mean'], ascending=False)

,n_splits,cluster_outliers,precision_mean,precision_se,recall_mean,recall_se,f1_mean,f1_se,time_mean,time_se
3,10,all,0.970766,0.000961,0.900290,0.001418,0.934191,0.000748,73.669478,0.648313
0,50,all,0.968803,0.000655,0.917085,0.000794,0.942232,0.000435,25.370456,0.098320
6,100,all,0.967798,0.000214,0.919595,0.000520,0.943080,0.000316,18.051203,0.032047
7,100,new,0.967798,0.000214,0.919595,0.000520,0.943080,0.000316,7.310424,0.045528
8,100,skip,0.967798,0.000214,0.919595,0.000520,0.943080,0.000316,7.272815,0.030110
1,50,new,0.781118,0.124943,0.917085,0.000794,0.765862,0.117503,14.124236,0.054112
2,50,skip,0.781118,0.124943,0.917085,0.000794,0.765861,0.117504,14.054986,0.066374
4,10,new,0.113677,0.094830,0.905598,0.001761,0.126363,0.089663,61.924965,0.523533
5,10,skip,0.113644,0.094834,0.905598,0.001761,0.126300,0.089670,61.510177,0.591904


For reference:

Basiline + 1by1 | t_s for 1by1 

* n_splits = 10 --> 30372 + 3375 | 61.696611
* n_splits = 50 --> 33072 + 675 | 14.362957
* n_splits = 100 --> 33409 + 338 | 7.342449

In [6]:
t_10 = 61.696611 / 3375
t_50 = 14.362957 / 675
t_100 = 7.342449 / 338

In [7]:
t_mean = (t_10 + t_50 + t_100) / 3

In [8]:
t_mean * 1000 #ms

20.427384680546428

In [9]:
# new columns 
# splits
# if n_splits = 10 --> "30372 + 3375"
# if n_splits = 50 --> "33072 + 675"
# if n_splits = 100 --> "33409 + 338"

def get_n_added(n_splits):
    if n_splits == 10:
        return 3375
    elif n_splits == 50:
        return 675
    elif n_splits == 100:
        return 338

def get_splits(n_splits):
    if n_splits == 10:
        return '30372+3375'
    elif n_splits == 50:
        return '33072+675'
    elif n_splits == 100:
        return '33409+338'

def get_splits_percentage(n_splits):
    if n_splits == 10:
        return 3375 / 30372
    elif n_splits == 50:
        return 675 / 33072
    elif n_splits == 100:
        return 338 / 33409

df_res['splits'] = df_res['n_splits'].apply(lambda x: get_splits(x))
df_res['splits_percentage'] = df_res['n_splits'].apply(lambda x: get_splits_percentage(x))
df_res['splits_percentage'] = df_res['splits_percentage'].apply(lambda x: round(x * 100, 2)).astype(str) + '%'
df_res['n_added'] = df_res['n_splits'].apply(lambda x: get_n_added(x))

In [10]:
df_res["precision"] = df_res["precision_mean"].round(4).astype(str) + "+/-" + df_res["precision_se"].round(4).astype(str)
df_res["recall"] = df_res["recall_mean"].round(4).astype(str) + "+/-" + df_res["recall_se"].round(4).astype(str)
df_res["f1"] = df_res["f1_mean"].round(4).astype(str) + "+/-" + df_res["f1_se"].round(4).astype(str)
df_res["time"] = df_res["time_mean"].round(2).astype(str) + "+/-" + df_res["time_se"].round(2).astype(str)

In [11]:
df_res.sort_values(by=["precision_mean", "f1_mean"], ascending=False)

,n_splits,cluster_outliers,precision_mean,precision_se,recall_mean,recall_se,f1_mean,f1_se,time_mean,time_se,splits,splits_percentage,n_added,precision,recall,f1,time
3,10,all,0.970766,0.000961,0.900290,0.001418,0.934191,0.000748,73.669478,0.648313,30372+3375,11.11%,3375,0.9708+/-0.001,0.9003+/-0.0014,0.9342+/-0.0007,73.67+/-0.65
0,50,all,0.968803,0.000655,0.917085,0.000794,0.942232,0.000435,25.370456,0.098320,33072+675,2.04%,675,0.9688+/-0.0007,0.9171+/-0.0008,0.9422+/-0.0004,25.37+/-0.1
6,100,all,0.967798,0.000214,0.919595,0.000520,0.943080,0.000316,18.051203,0.032047,33409+338,1.01%,338,0.9678+/-0.0002,0.9196+/-0.0005,0.9431+/-0.0003,18.05+/-0.03
7,100,new,0.967798,0.000214,0.919595,0.000520,0.943080,0.000316,7.310424,0.045528,33409+338,1.01%,338,0.9678+/-0.0002,0.9196+/-0.0005,0.9431+/-0.0003,7.31+/-0.05
8,100,skip,0.967798,0.000214,0.919595,0.000520,0.943080,0.000316,7.272815,0.030110,33409+338,1.01%,338,0.9678+/-0.0002,0.9196+/-0.0005,0.9431+/-0.0003,7.27+/-0.03
1,50,new,0.781118,0.124943,0.917085,0.000794,0.765862,0.117503,14.124236,0.054112,33072+675,2.04%,675,0.7811+/-0.1249,0.9171+/-0.0008,0.7659+/-0.1175,14.12+/-0.05
2,50,skip,0.781118,0.124943,0.917085,0.000794,0.765861,0.117504,14.054986,0.066374,33072+675,2.04%,675,0.7811+/-0.1249,0.9171+/-0.0008,0.7659+/-0.1175,14.05+/-0.07
4,10,new,0.113677,0.094830,0.905598,0.001761,0.126363,0.089663,61.924965,0.523533,30372+3375,11.11%,3375,0.1137+/-0.0948,0.9056+/-0.0018,0.1264+/-0.0897,61.92+/-0.52
5,10,skip,0.113644,0.094834,0.905598,0.001761,0.126300,0.089670,61.510177,0.591904,30372+3375,11.11%,3375,0.1136+/-0.0948,0.9056+/-0.0018,0.1263+/-0.0897,61.51+/-0.59


Observation:

* 1by1 works better with a small splits_percentage
* re-clustering all the outliers significantly increases the precision 
* clustering only the "new" outliers has no noticeable effect on the precision

Conclusion:

The LOW PRECISION is mainly driven by either an identity is not detected thus is
in the outliers (-1) clusters. Or the identity is "detected" but is in a "to be
discarded" clusters, i.e., a large cluster with obvious outliers / multiple
identities. 


* 1by1 is a good solution to add elements to the baseline only if a small number
  of picture is added to the dataset
* clustering only the new outliers is not useful in this case, however we
  believe depeding on the situation it could be useful, e.g., if we add 10+
  pictures of a new identity to the dataset
* This situation may not have been given enough attention in our test bench, and
  this could reflect a real life situation 
* Re-clustering the outliers is necessary if we want to maintain a high precision
  in the baseline

In [12]:
# rename columns cluster_outliers to "OB"
df_res.rename(columns={'cluster_outliers': 'OB', 'time': 'time [s]'}, inplace=True)
print(df_res.sort_values(by=["precision_mean", "f1_mean"], ascending=False).to_latex(index=False, columns=['splits', 'OB', 'precision', 'recall', 'f1', 'time [s]']))

\begin{tabular}{llllll}
\toprule
    splits &   OB &       precision &          recall &              f1 &     time [s] \\
\midrule
30372+3375 &  all &  0.9708+/-0.001 & 0.9003+/-0.0014 & 0.9342+/-0.0007 & 73.67+/-0.65 \\
 33072+675 &  all & 0.9688+/-0.0007 & 0.9171+/-0.0008 & 0.9422+/-0.0004 &  25.37+/-0.1 \\
 33409+338 &  all & 0.9678+/-0.0002 & 0.9196+/-0.0005 & 0.9431+/-0.0003 & 18.05+/-0.03 \\
 33409+338 &  new & 0.9678+/-0.0002 & 0.9196+/-0.0005 & 0.9431+/-0.0003 &  7.31+/-0.05 \\
 33409+338 & skip & 0.9678+/-0.0002 & 0.9196+/-0.0005 & 0.9431+/-0.0003 &  7.27+/-0.03 \\
 33072+675 &  new & 0.7811+/-0.1249 & 0.9171+/-0.0008 & 0.7659+/-0.1175 & 14.12+/-0.05 \\
 33072+675 & skip & 0.7811+/-0.1249 & 0.9171+/-0.0008 & 0.7659+/-0.1175 & 14.05+/-0.07 \\
30372+3375 &  new & 0.1137+/-0.0948 & 0.9056+/-0.0018 & 0.1264+/-0.0897 & 61.92+/-0.52 \\
30372+3375 & skip & 0.1136+/-0.0948 & 0.9056+/-0.0018 & 0.1263+/-0.0897 & 61.51+/-0.59 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_8046/1261789898.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_res.sort_values(by=["precision_mean", "f1_mean"], ascending=False).to_latex(index=False, columns=['splits', 'OB', 'precision', 'recall', 'f1', 'time [s]']))


In [13]:
df_res['t_p_face'] = df_res['time_mean'] / df_res['n_added']

In [15]:
print(df_res.sort_values(by=["precision_mean", "f1_mean"], ascending=False).to_latex(index=False, columns=['splits', 'OB', 'time [s]', 't_p_face']))

\begin{tabular}{lllr}
\toprule
    splits &   OB &     time [s] &  t\_p\_face \\
\midrule
30372+3375 &  all & 73.67+/-0.65 &  0.021828 \\
 33072+675 &  all &  25.37+/-0.1 &  0.037586 \\
 33409+338 &  all & 18.05+/-0.03 &  0.053406 \\
 33409+338 &  new &  7.31+/-0.05 &  0.021628 \\
 33409+338 & skip &  7.27+/-0.03 &  0.021517 \\
 33072+675 &  new & 14.12+/-0.05 &  0.020925 \\
 33072+675 & skip & 14.05+/-0.07 &  0.020822 \\
30372+3375 &  new & 61.92+/-0.52 &  0.018348 \\
30372+3375 & skip & 61.51+/-0.59 &  0.018225 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_8046/4217123112.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_res.sort_values(by=["precision_mean", "f1_mean"], ascending=False).to_latex(index=False, columns=['splits', 'OB', 'time [s]', 't_p_face']))
